In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
from config.config import MODEL_PATH, MODEL_CLASSES, MODEL_TYPE, DO_LOWER_CASE, BATCH_SIZE, DEVICE
from utils.load_data import *
from utils.utils import set_seed
import torch
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm
import pandas as pd


In [0]:
MODEL_PATH

'/home/tracy/zalo_qa/zalo_qa_zip/model/bert_multilingual_cased_vn_finetuned_3'

In [0]:
tokenizer = MODEL_CLASSES[MODEL_TYPE][2].from_pretrained(MODEL_PATH, do_lower_case=DO_LOWER_CASE)

In [0]:
import json
data = json.load(open("data/test.json",encoding='utf-8'))

In [0]:
examples = []
output_ids = []

for testcase in data:
    
    test_id = testcase["__id__"]
    text_a = testcase["question"]
    
    for paragraph in testcase["paragraphs"]:
        
        answer_id = paragraph["id"]
        output_ids.append({"test_id": test_id, "answer": answer_id})
        
        text_b = paragraph["text"]
        example = InputExample(guid="", text_a=text_a.lower(), text_b=text_b.lower(), label=0)
        examples.append(example)
        

In [0]:
cached_features_file = os.path.join(DATA_DIR, 'cached_testjson_{}_{}'.format(
                                    MODEL_TYPE, MAX_SEQ_LENGTH))

In [0]:
features = convert_examples_to_features(examples, tokenizer,
            cls_token_at_end=bool(MODEL_TYPE in ['xlnet']),            # xlnet has a cls token at the end
            cls_token=tokenizer.cls_token,
            sep_token=tokenizer.sep_token,
            cls_token_segment_id=2 if MODEL_TYPE in ['xlnet'] else 1,
            pad_on_left=bool(MODEL_TYPE in ['xlnet']),                 # pad on the left for xlnet
            pad_token_segment_id=4 if MODEL_TYPE in ['xlnet'] else 0)

logger.info("Saving features into cached file %s", cached_features_file)
torch.save(features, cached_features_file)

# Convert to Tensors and build dataset
all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

2019-10-24 09:58:06,787 - load_data.py - INFO - Writing example 0 of 2678
2019-10-24 09:58:06,792 - load_data.py - INFO - *** Example ***
2019-10-24 09:58:06,793 - load_data.py - INFO - guid: 
2019-10-24 09:58:06,795 - load_data.py - INFO - tokens: [CLS] To ##à nhà nào cao nhất Vi ##ệt Nam [SEP] The Landmark 81 là một toà nhà ch ##ọc trời trong tổ hợp dự án Vi ##nho ##mes T ##ân C ##ảng , một dự án có tổng mức đầu tư 40 . 000 tỷ đồng , do Công ty Cổ phần Đầu tư xây dựng T ##ân Liên Ph ##át thuộc Vi ##ng ##rou ##p làm chủ đầu tư . To ##à tháp cao 81 tầng , hiện tại là toà nhà cao nhất Vi ##ệt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . [SEP]
2019-10-24 09:58:06,796 - load_data.py - INFO - input_ids: 101 11469 10816 13265 27258 15341 13346 31826 30097 12645 102 10117 106969 12324 10331 10417 54591 13265 18643 51025 68121 10504 21217 16117 24467 15832 31826 47877 13546 157 15218 140 47129 117 10417 24467 15832 10601 23258 46417 11201 24257 10533 119 10259 34096 15049 117 1

2019-10-24 09:58:06,830 - load_data.py - INFO - tokens: [CLS] Mess ##i thuận chân nào [SEP] C ##ựu huấn luyện viên Bar ##cel ##ona , Pe ##p Guard ##iola , từng nhận xét rằng : " Mess ##i là cầu thủ duy nhất chạy với bóng còn nhanh hơn là chạy không bóng . " G ##i ##ống Mara ##dona , Mess ##i là một cầu thủ thuận chân trái . Anh thường sử dụng má ngoài chân trái khi giữ bóng , trong khi dùng má trong để dứt điểm , chu ##yền bóng và kiến tạo cho đồng đội . [SEP]
2019-10-24 09:58:06,831 - load_data.py - INFO - input_ids: 101 44006 10116 58629 28066 27258 102 140 71629 72170 52320 15202 20698 32942 12920 117 29005 10410 28739 63005 117 26258 16638 66331 16487 131 107 44006 10116 10331 19168 18755 38779 13346 38135 11182 22270 14674 41652 14789 10331 38135 11755 22270 119 107 144 10116 29028 48690 64674 117 44006 10116 10331 10417 19168 18755 58629 28066 57469 119 15212 16591 14808 15237 13267 25331 28066 57469 12072 25618 22270 117 10504 12072 22344 13267 10504 12460 101201 15509 117 10962

In [0]:
out_dir = "runs/finetune_3/"
config = MODEL_CLASSES[MODEL_TYPE][0].from_pretrained(out_dir, num_labels=2, finetuning_task="Racism")
model = MODEL_CLASSES[MODEL_TYPE][1].from_pretrained(out_dir, from_tf=False, config=config)

In [0]:
distill_sampler = SequentialSampler(dataset)
distill_dataloader = DataLoader(dataset, sampler=distill_sampler, batch_size=BATCH_SIZE)

In [0]:
model.to(DEVICE)
model.eval()
i = 2

out_sent = []
out_logit_0 = []
out_logit_1 = []

out_label = []
for batch in tqdm(distill_dataloader):
    batch = tuple(t.to(DEVICE) for t in batch)
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if MODEL_TYPE in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}

        outputs = model(**inputs)
        _, logits = outputs[:2]
        out_label.extend([0 if x[0] > x[1] else 1 for x in logits])

100%|██████████| 335/335 [00:33<00:00, 10.05it/s]


In [0]:
data_out = []
for k, l in zip(output_ids, out_label):
    if l == 1:
        data_out.append(k)
        

In [0]:
df = pd.DataFrame(data_out)
df.to_csv("data/submission_9.csv", index=False)